# 第四次课后练习

**负责助教：王瑞环**

<span style="color:red; font-weight:bold;">请将作业文件命名为 第四次课后练习+姓名+学号.ipynb, 例如 第四次课后练习+张三+1000000000.ipynb</span>

<span style="color:red; font-weight:bold;">在作业过程中觉得有心得或者自己拓展学习到有价值内容的，可以在文件名最后加一个#号。例如第四次课后练习+张三+1000000000+#.ipynb</span>

<span style="color:red; font-weight:bold;">本次课同时发布课后练习和选做题，提交时请注意区分提交通道</span>

# 第零部分 代码理解

请认真阅读代码，理解代码的功能，先写出预想的结果。运行并检验结果是否如预期。如果不如预期，请分析理解其中的原因

## **0.1** 类变量与实例变量

In [ ]:
class Config:
    data = {}
    
    def __init__(self, name):
        self.name = name
        self.data['key'] = 'default'

a = Config('a')
b = Config('b')
print(a.data is b.data)  
print(a.__class__.data is Config.data) 
a.data = {'key': 'custom'}
print(b.data['key'])

预期结果为
```bash
True
True 
default
```

实际与预期相符。

在`__init__`函数中，
```python
self.data['key'] = 'default'
```
由于实例字典中没有对应的`data`变量，所以这里Python会继续在`Config`的类字典中继续寻找对应的定义并进行修改。

1. `print(a.data is b.data)`: 这里`a`和`b`两次访问实例变量`data`均会回退到类变量，所以二者相同
2. `print(a.__class__.data is Config.data)`: `a.__class__.data`就是在访问`a`的类变量，也就是`Config.data`
3. `print(b.data['key'])`: 上一行中`a.data = {'key': 'custom'}`相当于直接赋值，与`__init__`函数中对`data`in-place modification不同，如果`a`没有实例变量`data`那么就创建一个，并不会影响类变量`data`

## **0.2** 单例模式

In [ ]:
class Singleton1:
    _instance = None
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls.value = 100
        return cls._instance

a = Singleton1()
a.value = 200
b = Singleton1()
print(a.value) 
print(b.value) 
print(Singleton1.value)   

预期结果为
```bash
200
200
100
```

结果与预期相符。

这里的`a`调用`Singleton1()`，由于此时`cls._instance`为`None`，所以会生成一个新的实例返回给`cls._instance`，并通过`a.value = 200`为这个实例创建实例变量`value = 200`；对于接下来的`b`而言，由于先前已经创建了`a`，所以这里返回的`cls._instance`与之前相同，所以实例变量`value`也是200，而对于`Singleton1`的类变量而言，其值为`__new__`函数中定义的100

In [ ]:
class Singleton2:
    _instance = None
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        cls.value = 100  
        return cls._instance

a = Singleton2()
a.value = 200
b = Singleton2()
print(a.value)
print(b.value)
print(Singleton2.value) 

预期结果为
```bash
200
200
100
```

实际与预期相符。这道题与前面的那一问改变了
```python
cls.value = 100
```

这一行代码不在`if`条件内，但是不影响输出的结果

In [ ]:
class Singleton3:
    _instance = None
    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        cls._instance.value = 100  
        return cls._instance

a = Singleton3()
a.value = 200
b = Singleton3()
print(a.value)
print(b.value)
print(Singleton3.value) 

预期结果为
```bash
100
100
AttributeError
```

实际与预期相符。这里的
```python
cls._instance.value = 100
```

相当于为返回的实例`cls._instance`创建了一个`value`并指定其值为100，所以这里想要输出的`Singleton3.value`不存在，因为`Singleton3`不存在类变量`value`

## **0.3** 函数闭包与工厂函数

In [ ]:
functions = []
for i in range(3):
    def func():
        return i
    functions.append(func)

print([f() for f in functions])

预期结果为
```bash
[2, 2, 2]
```

实际与预期相符。这里的函数`f()`只有在被调用时再会去调用`i`的值，所以所有的输出均为2

In [ ]:
functions = []
for i in range(3):
    def factory(i): 
        def func():
            return i 
        return func
    functions.append(factory(i)) 

print([f() for f in functions]) 

预期结果为
```bash
[0, 1, 2]
```

实际与预期相符。这里的不同之处是`i`此时为`factory`函数的参数，因此参数直接绑定，所以输出为对应`i`的值

## **0.4** 多进程编程

由于 Jupyter Notebook 无法直接运行多进程程序，我们采用一种迂回的方法：使用`%%writefile`将要运行的程序写入到一个`.py`文件中，再获取运行该文件所得到的结果

需要判断
- 输出是否稳定（即多次运行是否能保证输出结果一致）
  - 如果稳定，预测输出结果
  - 如果不稳定，预测所有输出结果的可能情况
- 代码大致的运行时间

In [ ]:
%%writefile multiprocessing_script.py
from multiprocessing import Pool
import time
import random

def square(x):
    time.sleep(1)
    return x*x

if __name__ == "__main__":
    
    with Pool(5) as p:
        start_time = time.time()
        result = p.map(square, range(20))
        print("Result:", result)
        end_time = time.time()
    print(f"All done, time taken: {end_time - start_time:.2f}")

In [ ]:
import subprocess
result = subprocess.run(
    ["python", "multiprocessing_script.py"], 
    capture_output=True, text=True)
print(result.stdout)

输出结果稳定，均为
```bash
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361]
```

预计的运行时间为4s，实际会略多于这个时间。这里每个方法的任务以及结果确定，不存在其他的输出情况。由于总共有20个任务，每个任务固定耗时1s，同时运行5个工作进程，所以预计运行时间为4s。

## **0.5** 生成器函数与 send() 操作

生成器函数（Generator Function）是一种通过 yield 语句逐步生成值的特殊函数。与普通函数不同，生成器函数通过 yield 暂停执行，保存当前状态，允许后续恢复执行。send() 是生成器的一个方法，用于向生成器内部传递数据，实现对生成器函数发送信息与控制。

示例：动态调整步长的计数器

In [ ]:
def dynamic_counter(start=0):
    current = start
    step = 1
    while True:
        # 通过yield返回当前值，并接收外部send的step值
        new_step = yield current
        if new_step is not None:  # 如果接收到新step，更新
            step = new_step
        current += step

In [ ]:
# 初始化生成器
counter = dynamic_counter(5)

# 启动生成器，获取初始值
print(next(counter))  

# 发送步长3，并获取下一个值
print(counter.send(3))  

# 继续迭代，使用当前步长3
print(next(counter))    

# 发送步长1，并获取下一个值
print(counter.send(1)) 

预期结果为
```bash
5
8
11
12
```

实际与预期相符。依据代码的注释，不难得到三次步长分别为3, 3, 1

## **0.6** `yield`递归生成器

In [ ]:
from collections.abc import Iterable
nested = [[[1], 2], 3, 4, [5, [[6, 7]]]]

In [ ]:
def flatten1(nested):
    for item in nested:
        if isinstance(item, Iterable) and not isinstance(item, (str, bytes)):
            for subitem in flatten1(item):
                yield subitem
        else:
            yield item

for i in flatten1(nested):
    print(i)

预期结果为
```bash
1
2
3
4
5
6
7
```

实际与预期相符。`flatten1`的作用相当于对每个`item`进行分析，如果可以直接输出就输出，如果是可迭代的就循环递归调用`flatten1`函数

In [ ]:
def flatten2(nested):
    for item in nested:
        if isinstance(item, Iterable) and not isinstance(item, (str, bytes)):
            yield from flatten2(item)
        else:
            yield item

for i in flatten2(nested):
    print(i)

预期结果为
```bash
1
2
3
4
5
6
7
```

实际与预期相符。这里的`yield from`写法与上个代码实现功能相同

In [ ]:
def flatten3(nested):
    for item in nested:
        if isinstance(item, Iterable) and not isinstance(item, (str, bytes)):
            flatten3(item)
        else:
            yield item

for i in flatten3(nested):
    print(i)

预期结果为
```bash
3
4
```

实际与预期相符。这里`flatten3`在循环递归调用后并没有`yield`，所有内部嵌套的`flatten3`得到的`yield`结果都被困在最深的那一层，并没有成功输出

In [ ]:
def flatten4(nested):
    for item in nested:
        if isinstance(item, Iterable) and not isinstance(item, (str, bytes)):
            yield flatten4(item)
        else:
            yield item

for i in flatten4(nested):
    print(i)

预期结果为
```bash
<generator object flatten4 at 0x...>
3
4
<generator object flatten4 at 0x...>
```

实际与预期相符。这里使用`yield`会对内部嵌套的部分返回generator object

# 第一部分 代码填空

## **1.1** 工厂模式

补充`SimpleFactory.create_product`函数，其接受一个字符串`product_type`作为参数，表示用户要求制作的产品
- 当`product_type`为`"A"`或`"Product A"`时，生产出一个`ProductA`对象并返回
- 当`product_type`为`"B"`或`"Product B"`时，生产出一个`ProductB`对象并返回
- 否则，抛出异常，表明商品无法制作

In [14]:
class ProductA:
    pass

class ProductB:
    pass

class SimpleFactory:
    @staticmethod
    def create_product(product_type: str):
        if product_type == "A" or product_type == "Product A":
            return ProductA()
        elif product_type == "B" or product_type == "Product B":
            return ProductB()
        else:
            raise ValueError("Invalid product type")

factory = SimpleFactory()

product_a1 = factory.create_product("A")
product_a2 = factory.create_product("Product A")
assert isinstance(product_a1, ProductA)
assert isinstance(product_a2, ProductA)
product_b1 = factory.create_product("B")
product_b2 = factory.create_product("Product B")
assert isinstance(product_b1, ProductB)
assert isinstance(product_b2, ProductB)


try: 
    product_c = factory.create_product("C")
except: 
    pass
else: 
    assert False

## **1.2** 观察者模式/发布-订阅模式

简单的来说，观察者模式就是由多位观察者(Observer)观察一个对象(Subject)，被观察对象发生了某种变化时，所有观察他的对象得到通知并被自动更新

如学生-教学网可以视为一组观察者-被观察对象：某课程在教学网上**发布**（公告、作业等）时，选课同学（**订阅/观察**该课程的学生）可以收到**通知**（作业截止日期、公告内容等），所以观察者模式又称为发布-订阅模式

GUI-鼠标也可以视为一组观察者-被观察对象：鼠标**发布**一个行为（点击、滚轮等）时，GUI的各个组件（可以认为是**订阅/观察**了该鼠标的行为）收到**通知**（鼠标点击的位置、左键还是右键等），并自动**更新**（如某个按钮被按下后，更新界面的显示内容）

<span style="color:red; font-weight:bold;">思考</span>：发布-订阅模式的更多使用场景？

本题要求实现一个简易的教学系统
- 学生可以选课或退课
- 教师可以在其负责的课程中发布信息
- 教师发布信息后，选课的学生可以收到信息

需要补充的内容包括
- `Course.add_student`函数：模拟学生选课，应保证已选课学生不会重复选
- `Course.remove_student`函数：模拟学生退课，应先检查的学生是否在选课名单里
- `Course.notify_students`函数：给所有学生发送通知
- `Teacher.post_message`函数：模拟教师给课程发布通知，应检查教师是否讲授该课程


In [ ]:
class Course:
    def __init__(self, course_name):
        self.course_name = course_name
        self.students = [] 
    
    def add_student(self, student):
        if student not in self.students:
            self.students.append(student)
    
    def remove_student(self, student):
        if student in self.students:
            self.students.remove(student)
    
    def notify_students(self, message):
        for student in self.students:
            student.receive_message(self.course_name, message)

class Student:
    def __init__(self, name, stu_id):
        self.name = name 
        self.stu_id = stu_id
    
    def receive_message(self, course_name, message):
        print(f"{self.name} 收到了 {course_name} 课程的通知: {message}")

class Teacher:
    def __init__(self, name, courses):
        self.name = name
        self.courses = courses
    
    def post_message(self, course, message):
        if course in self.courses:
            course.notify_students(message)
            


python_course = Course('Python 程序设计与数据科学导论')
ds_course = Course('数据结构与算法')

teacher_hu = Teacher('胡老师', [python_course])
teacher_wang = Teacher('王老师', [ds_course])

student1 = Student('小明', '001')
student2 = Student('小红', '002')
student3 = Student('小刚', '003')

# 模拟学生选课退课
python_course.add_student(student1)
python_course.add_student(student2)
python_course.add_student(student2)
python_course.add_student(student3)
ds_course.add_student(student2)  
ds_course.add_student(student3) 
ds_course.remove_student(student1)
assert len(python_course.students) == 3
assert len(ds_course.students) == 2

# 教师发布信息，应有三行输出
teacher_hu.post_message(python_course, "Python课程第一课：数据类型")
print()

# 教师发布信息，应有两行输出
teacher_wang.post_message(ds_course, "数据结构与算法第一课：线性表")
print()

# 教师发布信息，并非其负责课程，不应输出
teacher_wang.post_message(python_course, "Python课程第二课：函数")
print()

# 学生退课
python_course.remove_student(student3)
assert len(python_course.students) == 2

# 教师发布信息，应有两行输出
teacher_hu.post_message(python_course, "Python课程第二课：函数")
print()

## **1.3** `yield`生成器

实现求和计算器`SumCalculator`，使得
- 首次调用，打印问候语`"Give me integers to add them up"`并返回`None`
- 之后每次通过`send`传入
  - 整数：返回过去收到的所有整数的和
  - `None`：停止运行
  - 其他情况：打印`"I only accept integers"`，并返回现有的整数和

In [ ]:
def SumCalculator():
    total = 0
    
    print("Give me integers to add them up")
    value = yield None
    
    while True:
        if value is None:
            break
            
        if isinstance(value, int):
            total += value
        else:
            print("I only accept integers")
            
        value = yield total

repeater = SumCalculator()
assert next(repeater) is None   # 初次调用，应该打印问候语
assert repeater.send(1) == 1
assert repeater.send(2) == 3
assert repeater.send(3.) == 3   # 传入非整数，应该打印错误信息
assert repeater.send(3) == 6
assert repeater.send('4') == 6  # 传入非整数，应该打印错误信息
try:
    repeater.send(None)
except StopIteration:
    pass
else:
    assert False